In [1]:
import pandas as pd
import requests
import re

In [2]:
url_capas="https://situ.sedatu.gob.mx/wa/publico/getCapas/"

In [3]:
response = requests.get(url_capas)
response.status_code

200

In [4]:
capas=pd.DataFrame(response.json()).rename(columns=str.lower)
#Dejar solo la columna id_capa
capas=capas[['titulo','nombre_capa','id_capa']]
#Sort 
capas=capas.sort_values(by='id_capa')
capas

,titulo,nombre_capa,id_capa
1988,Limite Estatal,geonode:inegi_00ent_4326,2
1096,Limite Municipal,geonode:inegi_00mun_4326,3
277,Hidrografía,geonode:inegi_RH00_ha_4326,5
666,Localidades Rurales Amanzanadas,geonode:inegi_00l_ra_4326,6
227,Localidades Urbanas,geonode:inegi_00l_ua_4326,7
...,...,...,...
1899,PTO ZSRM. Eje 1. Gestión efectiva y gobernanza...,geonode:lineas_e_gestion_efectiva_y_gobernanza...,2258
87,PTO ZSRM. Eje 1. Gestión efectiva y gobernanza...,geonode:poligonos_e_gestion_efectiva_y_goberna...,2259
479,PTO ZSRM. Eje 4. Economía regional. Puntos,geonode:puntos_e_economia_regional_pto_zsrm_2023,2260
1739,PTO ZSRM. Eje 4. Economía regional. Líneas,geonode:lineas_e_economia_regional_pto_zsrm_2023,2261


In [5]:
#Función para extraer todos los tags de un texto
def extract_all_tags(text, tag):
    pattern = fr"<{tag}>(.*?)</{tag}>"
    return re.findall(pattern, text, re.DOTALL)

In [6]:
# Lista para almacenar los metadatos
metadata_list = []

# lista de id_capa
id_capa_list = capas['id_capa'].tolist()

# Iterate over each id_capa
for id_capa in id_capa_list:
    url_metadatos = f"https://situ.sedatu.gob.mx/wa/publico/metadatos/consultaMetadatos/{id_capa}"
    
    # Make the request
    try:
        response = requests.get(url_metadatos)
        data = response.json()
        
        # Check if data is empty
        if not data:
            print(f"No hay metadato para la capa {id_capa}. Saltando al siguiente...")
            continue
        
        # Extract results
        resultados = data[0][3] if data[0][3] else []
        
        # Check if 'resultados' is a string and convert it to a list
        if isinstance(resultados, str):
            resultados = [resultados]

        # Extract data from the results
        for result in resultados:
            result_cleaned = result.replace("\\n", "").replace("\\r", "").strip()
            extracted_title = extract_all_tags(result_cleaned, "title")[0] if extract_all_tags(result_cleaned, "title") else None
            extracted_purpose = extract_all_tags(result_cleaned, "purpose")[0] if extract_all_tags(result_cleaned, "purpose") else None
            extract_abstract = extract_all_tags(result_cleaned, "abstract")[0] if extract_all_tags(result_cleaned, "abstract") else None
            extract_language = extract_all_tags(result_cleaned, "MD_DataIdentification_language")[0] if extract_all_tags(result_cleaned, "MD_DataIdentification_language") else None
            extract_topic = extract_all_tags(result_cleaned, "principalTopic")[0] if extract_all_tags(result_cleaned, "principalTopic") else None
            extract_group_category = extract_all_tags(result_cleaned, "groupCategory")[0] if extract_all_tags(result_cleaned, "groupCategory") else None
            extract_keywords = extract_all_tags(result_cleaned, "keyword")  
            extract_thesauros = extract_all_tags(result_cleaned, "thesaurusName")[0] if extract_all_tags(result_cleaned, "thesaurusName") else None
            extract_edition=extract_all_tags(result_cleaned, "edition")[0] if extract_all_tags(result_cleaned, "edition") else None
            extract_presentation_form=extract_all_tags(result_cleaned, "presentationForm")[0] if extract_all_tags(result_cleaned, "presentationForm") else None
            extract_url=extract_all_tags(result_cleaned, "CI_OnlineResource_linkage")[0] if extract_all_tags(result_cleaned, "CI_OnlineResource_linkage") else None
            extract_url_description=extract_all_tags(result_cleaned, "CI_OnlineResource_description")[0] if extract_all_tags(result_cleaned, "CI_OnlineResource_description") else None
            extract_maintenace_frequency=extract_all_tags(result_cleaned, "maintenanceAndUpdateFrequency")[0] if extract_all_tags(result_cleaned, "maintenanceAndUpdateFrequency") else None
            extract_caracteres=extract_all_tags(result_cleaned, "MD_DataIdentification_characterSet")[0] if extract_all_tags(result_cleaned, "MD_DataIdentification_characterSet") else None
            extract_graph_file=extract_all_tags(result_cleaned, "MD_DataIdentification_characterSet")[0] if extract_all_tags(result_cleaned, "MD_DataIdentification_characterSet") else None
            extract_specific_use=extract_all_tags(result_cleaned, "specuse")[0] if extract_all_tags(result_cleaned, "specuse") else None
            extract_date_ref=extract_all_tags(result_cleaned, "date")[0] if extract_all_tags(result_cleaned, "date") else None
            extract_date_type=extract_all_tags(result_cleaned, "dateType")[0] if extract_all_tags(result_cleaned, "dateType") else None
            extract_date_create=extract_all_tags(result_cleaned, "date_creation")[0] if extract_all_tags(result_cleaned, "date_creation") else None
            extract_date_begin=extract_all_tags(result_cleaned, "date_begin")[0] if extract_all_tags(result_cleaned, "date_begin") else None
            extract_date_end=extract_all_tags(result_cleaned, "date_end")[0] if extract_all_tags(result_cleaned, "date_end") else None
            extract_nombre_insumo=extract_all_tags(result_cleaned, "inpname")[0] if extract_all_tags(result_cleaned, "inpname") else None
            extract_persona_contacto=extract_all_tags(result_cleaned, "CI_ResponsibleParty_individualName")[0] if extract_all_tags(result_cleaned, "CI_ResponsibleParty_individualName") else None
            extract_organizacion_persona=extract_all_tags(result_cleaned, "CI_ResponsibleParty_organisationName")[0] if extract_all_tags(result_cleaned, "CI_ResponsibleParty_organisationName") else None
            extract_puesto_contacto=extract_all_tags(result_cleaned, "CI_ResponsibleParty_positionName")[0] if extract_all_tags(result_cleaned, "CI_ResponsibleParty_positionName") else None
            extract_telephone=extract_all_tags(result_cleaned, "CI_ResponsibleParty_voice")[0] if extract_all_tags(result_cleaned, "CI_ResponsibleParty_voice") else None
            extract_fax=extract_all_tags(result_cleaned, "CI_ResponsibleParty_facsimile")[0] if extract_all_tags(result_cleaned, "CI_ResponsibleParty_facsimile") else None
            extract_direccion_contacto=extract_all_tags(result_cleaned, "CI_ResponsibleParty_deliveryPoint")[0] if extract_all_tags(result_cleaned, "CI_ResponsibleParty_deliveryPoint") else None
            extract_ciudad_contacto=extract_all_tags(result_cleaned, "CI_ResponsibleParty_city")[0] if extract_all_tags(result_cleaned, "CI_ResponsibleParty_city") else None
            extract_area_admin=extract_all_tags(result_cleaned, "CI_ResponsibleParty_administrativeArea")[0] if extract_all_tags(result_cleaned, "CI_ResponsibleParty_administrativeArea") else None
            extract_cp=extract_all_tags(result_cleaned, "CI_ResponsibleParty_postalCode")[0] if extract_all_tags(result_cleaned, "CI_ResponsibleParty_postalCode") else None
            extract_country=extract_all_tags(result_cleaned, "CI_ResponsibleParty_country")[0] if extract_all_tags(result_cleaned, "CI_ResponsibleParty_country") else None
            extract_email=extract_all_tags(result_cleaned, "CI_ResponsibleParty_electronicMailAddress")[0] if extract_all_tags(result_cleaned, "CI_ResponsibleParty_electronicMailAddress") else None
            extract_url_contacto=extract_all_tags(result_cleaned, "CI_ResponsibleParty_linkage")[0] if extract_all_tags(result_cleaned, "CI_ResponsibleParty_linkage") else None
            extract_rol_contacto=extract_all_tags(result_cleaned, "CI_ResponsibleParty_role")[0] if extract_all_tags(result_cleaned, "CI_ResponsibleParty_role") else None
            extract_coordenada_este=extract_all_tags(result_cleaned, "eastBoundLongitude")[0] if extract_all_tags(result_cleaned, "eastBoundLongitude") else None
            extract_coordenada_oeste=extract_all_tags(result_cleaned, "westBoundLongitude")[0] if extract_all_tags(result_cleaned, "westBoundLongitude") else None
            extract_coordenada_norte=extract_all_tags(result_cleaned, "northBoundLatitude")[0] if extract_all_tags(result_cleaned, "northBoundLatitude") else None
            extract_coordenada_sur=extract_all_tags(result_cleaned, "southBoundLatitude")[0] if extract_all_tags(result_cleaned, "southBoundLatitude") else None
            extract_representacion=extract_all_tags(result_cleaned, "spatialRepresentationType")[0] if extract_all_tags(result_cleaned, "spatialRepresentationType") else None
            extract_descrip_atributos=extract_all_tags(result_cleaned, "schemaAscii")[0] if extract_all_tags(result_cleaned, "schemaAscii") else None
            extract_detalle_atributos=extract_all_tags(result_cleaned, "entity_detail")[0] if extract_all_tags(result_cleaned, "entity_detail") else None
            extract_lambert=extract_all_tags(result_cleaned, "lambertc")
            extract_met_cod=extract_all_tags(result_cleaned, "plance")[0] if extract_all_tags(result_cleaned, "plance") else None
            extract_ordenada=extract_all_tags(result_cleaned, "ordres")[0] if extract_all_tags(result_cleaned, "ordres") else None
            extract_absisa=extract_all_tags(result_cleaned, "absres")[0] if extract_all_tags(result_cleaned, "absres") else None
            extract_nom_datum=extract_all_tags(result_cleaned, "horizdn")[0] if extract_all_tags(result_cleaned, "horizdn") else None
            extract_elipsoide=extract_all_tags(result_cleaned, "ellips")[0] if extract_all_tags(result_cleaned, "ellips") else None
            extract_semiaxis=extract_all_tags(result_cleaned, "semiaxis")[0] if extract_all_tags(result_cleaned, "semiaxis") else None
            extract_denom=extract_all_tags(result_cleaned, "denflat")[0] if extract_all_tags(result_cleaned, "denflat") else None
            extract_enunciado=extract_all_tags(result_cleaned, "statement")[0] if extract_all_tags(result_cleaned, "statement") else None
            extract_proceso=extract_all_tags(result_cleaned, "LI_ProcessStep_description")[0] if extract_all_tags(result_cleaned, "LI_ProcessStep_description") else None
            extract_fuente=extract_all_tags(result_cleaned, "LI_Source_description")[0] if extract_all_tags(result_cleaned, "LI_Source_description") else None
            extract_rest_uso=extract_all_tags(result_cleaned, "accessConstraints")[0] if extract_all_tags(result_cleaned, "accessConstraints") else None
            extract_rest_acceso=extract_all_tags(result_cleaned, "useConstraints")[0] if extract_all_tags(result_cleaned, "useConstraints") else None
            extract_other=extract_all_tags(result_cleaned, "otherConstraints")[0] if extract_all_tags(result_cleaned, "otherConstraints") else None
            extract_name_soft=extract_all_tags(result_cleaned, "name")[0] if extract_all_tags(result_cleaned, "name") else None
            extract_version=extract_all_tags(result_cleaned, "version")[0] if extract_all_tags(result_cleaned, "version") else None
            extract_nom_metadatos=extract_all_tags(result_cleaned, "metadataStandardName")[0] if extract_all_tags(result_cleaned, "metadataStandardName") else None
            extract_version_metadatos=extract_all_tags(result_cleaned, "metadataStandardVersion")[0] if extract_all_tags(result_cleaned, "metadataStandardVersion") else None
            extract_language_metadatos=extract_all_tags(result_cleaned, "Inf_Metadata_language")[0] if extract_all_tags(result_cleaned, "Inf_Metadata_language") else None
            extract_fecha_metadatos=extract_all_tags(result_cleaned, "dateStamp")[0] if extract_all_tags(result_cleaned, "dateStamp") else None
            
            
            
            # Append the data to the metadata list
            metadata_list.append({
                "id_capa": id_capa,
                "titulo_capa": extracted_title,
                "purpose": extracted_purpose,
                "abstract": extract_abstract,
                "language": extract_language,
                "topic": extract_topic,
                "group_category": extract_group_category,
                "keywords": extract_keywords,
                "thesaurus": extract_thesauros,
                "edition": extract_edition,
                "presentation_form": extract_presentation_form,
                "url": extract_url,
                "url_description": extract_url_description,
                "maintenance_frequency": extract_maintenace_frequency,
                "caracteres": extract_caracteres,
                "graph_file": extract_graph_file,
                "specific_use": extract_specific_use,
                "fecha_ref": extract_date_ref,
                "tipo_fecha": extract_date_type,
                "fecha_creacion": extract_date_create,
                "fecha_inicio": extract_date_begin,
                "fecha_fin": extract_date_end,
                "nombre_insumo": extract_nombre_insumo,
                "persona_contacto": extract_persona_contacto,
                "organizacion_contacto": extract_organizacion_persona,
                "puesto_contacto": extract_puesto_contacto,
                "telefono_contacto": extract_telephone,
                "fax_contacto": extract_fax,
                "direccion_contacto": extract_direccion_contacto,
                "ciudad_contacto": extract_ciudad_contacto,
                "area_admin": extract_area_admin,
                "cp": extract_cp,
                "pais": extract_country,
                "email": extract_email,
                "url_contacto": extract_url_contacto,
                "rol_contacto": extract_rol_contacto,
                "coordenada_este": extract_coordenada_este,
                "coordenada_oeste": extract_coordenada_oeste,
                "coordenada_norte": extract_coordenada_norte,
                "coordenada_sur": extract_coordenada_sur,
                "representacion": extract_representacion,
                "descrip_atributos": extract_descrip_atributos,
                "detalle_atributos": extract_detalle_atributos,
                "lambert": extract_lambert,
                "met_cod": extract_met_cod,
                "ordenada": extract_ordenada,
                "absisa": extract_absisa,
                "nom_datum": extract_nom_datum,
                "elipsoide": extract_elipsoide,
                "semieje_mayor": extract_semiaxis,
                "denominador": extract_denom,
                "enunciado": extract_enunciado,
                "proceso": extract_proceso,
                "fuente": extract_fuente,
                "rest_uso": extract_rest_uso,
                "rest_acceso": extract_rest_acceso,
                "distribucion": extract_other,
                "name_soft": extract_name_soft,
                "version": extract_version,
                "nom_metadatos": extract_nom_metadatos,
                "version_metadatos": extract_version_metadatos,
                "language_metadatos": extract_language_metadatos,
                "fecha_metadatos": extract_fecha_metadatos
                
            })
    
    except requests.exceptions.RequestException as e:
        print(f"Error obteniendo datos de la capa {id_capa}: {e}")
        continue


No hay metadato para la capa 5. Saltando al siguiente...
No hay metadato para la capa 6. Saltando al siguiente...
No hay metadato para la capa 9. Saltando al siguiente...
No hay metadato para la capa 10. Saltando al siguiente...
No hay metadato para la capa 11. Saltando al siguiente...
No hay metadato para la capa 13. Saltando al siguiente...
No hay metadato para la capa 15. Saltando al siguiente...
No hay metadato para la capa 16. Saltando al siguiente...
No hay metadato para la capa 17. Saltando al siguiente...
No hay metadato para la capa 18. Saltando al siguiente...
No hay metadato para la capa 19. Saltando al siguiente...
No hay metadato para la capa 20. Saltando al siguiente...
No hay metadato para la capa 22. Saltando al siguiente...
No hay metadato para la capa 24. Saltando al siguiente...
No hay metadato para la capa 25. Saltando al siguiente...
No hay metadato para la capa 26. Saltando al siguiente...
No hay metadato para la capa 27. Saltando al siguiente...
No hay metadato p

In [7]:
metadata_df=pd.DataFrame(metadata_list)
print("Número de columnas: ",metadata_df.shape[1]-1)
metadata_df

Número de columnas:  62


,id_capa,titulo_capa,purpose,abstract,language,topic,group_category,keywords,thesaurus,edition,...,fuente,rest_uso,rest_acceso,distribucion,name_soft,version,nom_metadatos,version_metadatos,language_metadatos,fecha_metadatos
0,2,"MARCO GEOESTADÍSTICO, CENSO DE POBLACIÓN Y VIV...",\r\nProporcionar a las unidades de Estado y a ...,\r\nEl Marco Geoestadístico (MG) es un product...,ES - Español.,11. Localización: Información posicional y ser...,"6. Grupo de datos de límites costeros, interna...","[AGEB - Área Geoestadística Básica, AGEE - Áre...",Manuales definidos para uso específico del Mar...,,...,\r\nÁMBITO LOCALIDADES URBANAS Y LOCALIDADES R...,8. Acceso al público a través de internet y lo...,8. Ninguna.,El Instituto Nacional de Estadística y Geograf...,Shape,Arc Map 10.8.1,ISO 19115 Información Geográfica - Metadatos (...,2003 (1.0).,ES - Español.,2021-01-15
1,3,"MARCO GEOESTADÍSTICO, CENSO DE POBLACIÓN Y VIV...",\nProporcionar a las unidades de Estado y a la...,\nEl Marco Geoestadístico (MG) es un producto ...,ES - Español.,11. Localización: Información posicional y ser...,"6. Grupo de datos de límites costeros, interna...","[AGEB - Área Geoestadística Básica, AGEE - Áre...",Manuales definidos para uso específico del Mar...,,...,\nÁMBITO LOCALIDADES URBANAS Y LOCALIDADES RUR...,8. Acceso al público a través de internet y lo...,8. Ninguna.,El Instituto Nacional de Estadística y Geograf...,Shape,Arc Map 10.8.1,ISO 19115 Información Geográfica - Metadatos (...,2003 (1.0).,ES - Español.,2021-01-15
2,7,"MARCO GEOESTADÍSTICO, CENSO DE POBLACIÓN Y VIV...",\r\nProporcionar a las unidades de Estado y a ...,\r\nEl Marco Geoestadístico (MG) es un product...,ES - Español.,11. Localización: Información posicional y ser...,"6. Grupo de datos de límites costeros, interna...","[AGEB - Área Geoestadística Básica, AGEE - Áre...",Manuales definidos para uso específico del Mar...,,...,\r\nÁMBITO LOCALIDADES URBANAS Y LOCALIDADES R...,8. Acceso al público a través de internet y lo...,8. Ninguna.,El Instituto Nacional de Estadística y Geograf...,Shape,Arc Map 10.8.1,ISO 19115 Información Geográfica - Metadatos (...,2003 (1.0).,ES - Español.,2021-01-15
3,8,Red Nacional de Caminos (RNC) 2020 (00_RNC_IMT...,Proporcionar a las Unidades del Estado y a la ...,La Red Nacional de Caminos es una red vial mod...,ES - Español.,11. Localización: Información posicional y ser...,4. Grupo de datos topográficos: Subdivisión de...,"[Carretera, Carreteras Pavimentadas, Estados U...",Diccionario de Datos de la Red Nacional de Cam...,2020,...,INFORMACIÓN BASE: Red Nacional de Caminos 2019...,6. Derechos de propiedad intelectual: Es el de...,8. Ninguna,SEDATU no se hace responsable por el uso que s...,"Shapefile, dbase",QGIS 3.16.0-Hannover,ISO 19115 Información Geográfica - Metadatos (...,2003 (1.0).,ES - Español.,2021-01-21
4,12,Áreas Naturales Protegidas Federales de la Rep...,Proporcionar datos espaciales de las Áreas Nat...,Datos espaciales de las Áreas Naturales Proteg...,ES - Español.,5. Medio ambiente: Recursos medio ambientale...,1. Grupo de datos de recursos naturales y clim...,"[Área Natural Protegida, Área de Protección de...",,,...,"CONANP, (2017/11/01), Áreas Naturales Protegid...",,,SEDATU no se hace responsable por el uso que s...,Shapefile.,SEDATU: QGIS 3.16.0-Hannover,ISO 19115 Información Geográfica - Metadatos (...,2003 (1.0).,ES - Español.,2021-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1502,2223,Zonificación Primaria del Programa Municipal d...,La zonificación primaria determina las áreas q...,Para la elaboración del shapefile se definiero...,ES - Español.,12. Planeamiento catastral: Información utiliz...,3. Grupo de datos catastrales: Subdivisión de ...,[Planificación del territorio y catastral],Programa Municipal de Ordenamiento Territorial...,2024.,...,Programa Municipal de Ordenamiento Territorial...,8. Aceso al público a través del Sistema de I...,8. Restricciones de uso: Para consulta y 

In [8]:
#Cruzar con el DataFrame de capas
metadata_final=pd.merge(capas,metadata_df,on='id_capa')
metadata_final

,titulo,nombre_capa,id_capa,titulo_capa,purpose,abstract,language,topic,group_category,keywords,...,fuente,rest_uso,rest_acceso,distribucion,name_soft,version,nom_metadatos,version_metadatos,language_metadatos,fecha_metadatos
0,Limite Estatal,geonode:inegi_00ent_4326,2,"MARCO GEOESTADÍSTICO, CENSO DE POBLACIÓN Y VIV...",\r\nProporcionar a las unidades de Estado y a ...,\r\nEl Marco Geoestadístico (MG) es un product...,ES - Español.,11. Localización: Información posicional y ser...,"6. Grupo de datos de límites costeros, interna...","[AGEB - Área Geoestadística Básica, AGEE - Áre...",...,\r\nÁMBITO LOCALIDADES URBANAS Y LOCALIDADES R...,8. Acceso al público a través de internet y lo...,8. Ninguna.,El Instituto Nacional de Estadística y Geograf...,Shape,Arc Map 10.8.1,ISO 19115 Información Geográfica - Metadatos (...,2003 (1.0).,ES - Español.,2021-01-15
1,Limite Municipal,geonode:inegi_00mun_4326,3,"MARCO GEOESTADÍSTICO, CENSO DE POBLACIÓN Y VIV...",\nProporcionar a las unidades de Estado y a la...,\nEl Marco Geoestadístico (MG) es un producto ...,ES - Español.,11. Localización: Información posicional y ser...,"6. Grupo de datos de límites costeros, interna...","[AGEB - Área Geoestadística Básica, AGEE - Áre...",...,\nÁMBITO LOCALIDADES URBANAS Y LOCALIDADES RUR...,8. Acceso al público a través de internet y lo...,8. Ninguna.,El Instituto Nacional de Estadística y Geograf...,Shape,Arc Map 10.8.1,ISO 19115 Información Geográfica - Metadatos (...,2003 (1.0).,ES - Español.,2021-01-15
2,Localidades Urbanas,geonode:inegi_00l_ua_4326,7,"MARCO GEOESTADÍSTICO, CENSO DE POBLACIÓN Y VIV...",\r\nProporcionar a las unidades de Estado y a ...,\r\nEl Marco Geoestadístico (MG) es un product...,ES - Español.,11. Localización: Información posicional y ser...,"6. Grupo de datos de límites costeros, interna...","[AGEB - Área Geoestadística Básica, AGEE - Áre...",...,\r\nÁMBITO LOCALIDADES URBANAS Y LOCALIDADES R...,8. Acceso al público a través de internet y lo...,8. Ninguna.,El Instituto Nacional de Estadística y Geograf...,Shape,Arc Map 10.8.1,ISO 19115 Información Geográfica - Metadatos (...,2003 (1.0).,ES - Español.,2021-01-15
3,Red Nacional de Caminos,geonode:inegi_rnc_redvial_4326,8,Red Nacional de Caminos (RNC) 2020 (00_RNC_IMT...,Proporcionar a las Unidades del Estado y a la ...,La Red Nacional de Caminos es una red vial mod...,ES - Español.,11. Localización: Información posicional y ser...,4. Grupo de datos topográficos: Subdivisión de...,"[Carretera, Carreteras Pavimentadas, Estados U...",...,INFORMACIÓN BASE: Red Nacional de Caminos 2019...,6. Derechos de propiedad intelectual: Es el de...,8. Ninguna,SEDATU no se hace responsable por el uso que s...,"Shapefile, dbase",QGIS 3.16.0-Hannover,ISO 19115 Información Geográfica - Metadatos (...,2003 (1.0).,ES - Español.,2021-01-21
4,Areas Naturales Protegidas,geonode:conanp_182ANP_4326,12,Áreas Naturales Protegidas Federales de la Rep...,Proporcionar datos espaciales de las Áreas Nat...,Datos espaciales de las Áreas Naturales Proteg...,ES - Español.,5. Medio ambiente: Recursos medio ambientale...,1. Grupo de datos de recursos naturales y clim...,"[Área Natural Protegida, Área de Protección de...",...,"CONANP, (2017/11/01), Áreas Naturales Protegid...",,,SEDATU no se hace responsable por el uso que s...,Shapefile.,SEDATU: QGIS 3.16.0-Hannover,ISO 19115 Información Geográfica - Metadatos (...,2003 (1.0).,ES - Español.,2021-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1514,Zonificación Primaria del Programa Municipal d...,geonode:c25010_zonificacion_primaria_pmdu_2024,2223,Zonificación Primaria del Programa Municipal d...,La zonificación primaria determina las áreas q...,Para la elaboración del shapefile se definiero...,ES - Español.,12. Planeamiento catastral: Información utiliz...,3. Grupo de datos catastrales: Subdivisión de ...,[Planificación del territorio y catastral],...,Programa Municipal de Ordenamiento Territorial...,8. Aceso al público a

In [9]:
print("Porcentaje de capas con metadatos: ",metadata_df.shape[0]/capas.shape[0]*100)

Porcentaje de capas con metadatos:  74.78908188585608
